In [18]:
import torchaudio

train_dataset = torchaudio.datasets.LIBRISPEECH("./", url="dev-clean", download=False)
test_dataset = torchaudio.datasets.LIBRISPEECH("./", url="test-clean", download=False)

In [19]:
train_dataset[0]

(tensor([[0.0024, 0.0021, 0.0020,  ..., 0.0004, 0.0006, 0.0010]]),
 16000,
 'MISTER QUILTER IS THE APOSTLE OF THE MIDDLE CLASSES AND WE ARE GLAD TO WELCOME HIS GOSPEL',
 1272,
 128104,
 0)

In [127]:
import torch.nn as nn
import torch as torch


mfcc_transform = torchaudio.transforms.MFCC(sample_rate=16000,n_mfcc=13)
def data_processing(data,):
    spectrograms = []
    mfcc=[]
    labels=[]
    input_lengths = []
    for (waveform, _,ll, _, _, _) in data:
        mfcc.append(mfcc_transform(waveform).squeeze(0).transpose(0, 1))
        labels.append(ll.lower())
    mfcc = nn.utils.rnn.pad_sequence(mfcc, batch_first=True,zwe)
    return mfcc,labels

In [128]:
mfcc,text=data_processing(train_dataset)

In [130]:
len(text)

2703

In [131]:
mfcc.shape

torch.Size([2703, 2612, 13])

In [172]:
torch.save(mfcc, 'Mfcc_features.pt')
torch.save(text,'Labels.pt')
# torch.load('Mfcc_features.pt')

##  Working with word2vec and fastText

In [174]:
import numpy as np
import os
from random import shuffle
import re
import urllib.request
import zipfile
import lxml.etree
#download the data
urllib.request.urlretrieve("https://wit3.fbk.eu/get.php?path=XML_releases/xml/ted_en-20160408.zip&filename=ted_en-20160408.zip", filename="ted_en-20160408.zip")
# extract subtitle
with zipfile.ZipFile('ted_en-20160408.zip', 'r') as z:
    doc = lxml.etree.parse(z.open('ted_en-20160408.xml', 'r'))
input_text = '\n'.join(doc.xpath('//content/text()'))

In [177]:
input_text[0:100]

"Here are two reasons companies fail: they only do more of the same, or they only do what's new.\nTo m"

In [178]:
# remove parenthesis 
input_text_noparens = re.sub(r'\([^)]*\)', '', input_text)
# store as list of sentences
sentences_strings_ted = []
for line in input_text_noparens.split('\n'):
    m = re.match(r'^(?:(?P<precolon>[^:]{,20}):)?(?P<postcolon>.*)$', line)
    sentences_strings_ted.extend(sent for sent in m.groupdict()['postcolon'].split('.') if sent)
# store as list of lists of words
sentences_ted = []
for sent_str in sentences_strings_ted:
    tokens = re.sub(r"[^a-z0-9]+", " ", sent_str.lower()).split()
    sentences_ted.append(tokens)

In [180]:
len(sentences_ted)

266694

In [181]:
sentences_ted[0]

['here',
 'are',
 'two',
 'reasons',
 'companies',
 'fail',
 'they',
 'only',
 'do',
 'more',
 'of',
 'the',
 'same',
 'or',
 'they',
 'only',
 'do',
 'what',
 's',
 'new']

In [182]:

from gensim.models import Word2Vec

model_ted = Word2Vec(sentences=Sen, size=10, window=3, min_count=5, workers=4, sg=0)
model_ted = Word2Vec(sentences=sentences_ted, size=10, window=3, min_count=5, workers=4, sg=0)

In [184]:
model_ted.wv.most_similar('man')

[('woman', 0.9832761287689209),
 ('girl', 0.9495598077774048),
 ('boy', 0.9316831231117249),
 ('lady', 0.9284731149673462),
 ('friend', 0.9236975908279419),
 ('guy', 0.9091511368751526),
 ('child', 0.896727979183197),
 ('brother', 0.8794185519218445),
 ('father', 0.8767870664596558),
 ('daughter', 0.876102864742279)]

In [186]:
from gensim.models import FastText
model_ted = FastText(sentences_ted, size=10, window=3, min_count=5, workers=4,sg=1)

In [187]:
model_ted.wv.most_similar('Gastroenteritis')

[('borderless', 0.9880135655403137),
 ('benevolent', 0.986836314201355),
 ('initiative', 0.9858351945877075),
 ('delaware', 0.985022783279419),
 ('diplomat', 0.9812185764312744),
 ('censorship', 0.980431079864502),
 ('idealab', 0.9802150130271912),
 ('anagnorisis', 0.9793952703475952),
 ('aeronautics', 0.9790065884590149),
 ('compartmentalized', 0.9787019491195679)]

In [189]:
a=model_ted.wv['car']

In [190]:
a

array([-0.696699  ,  1.0907794 ,  0.05345207, -0.90658665, -0.69240624,
       -0.26275662, -0.11506832, -0.6883314 , -0.11154088, -0.1074451 ],
      dtype=float32)

In [191]:
len(a)

10

In [195]:
model_ted.wv.evaluate_word_pairs('wordsim353.tsv')

((0.31209907796701986, 8.079138223651747e-09),
 SpearmanrResult(correlation=0.3112698232108314, pvalue=8.889208127817677e-09),
 7.365439093484419)

In [1]:
import torch
mfcc = torch.load('Mfcc_feature.pt')

In [2]:
mfcc.shape

AttributeError: 'dict' object has no attribute 'shape'

In [27]:
import numpy as np
input=[]
for m in mfcc['mfcc']:
    input.append(m)

In [28]:
input=np.asarray(input).transpose(0,2,1)

(20, 400, 13)

In [30]:
mfcc['mfcc'][0]

array([[-676.4033  , -655.20734 , -652.4823  , ...,    0.      ,
           0.      ,    0.      ],
       [  60.13748 ,   62.161404,   53.713818, ...,    0.      ,
           0.      ,    0.      ],
       [ -16.710009,  -26.98094 ,  -40.093697, ...,    0.      ,
           0.      ,    0.      ],
       ...,
       [  15.198107,   19.322918,   18.376934, ...,    0.      ,
           0.      ,    0.      ],
       [ -23.876957,  -16.92124 ,  -28.317642, ...,    0.      ,
           0.      ,    0.      ],
       [  28.821056,   20.086813,    9.370873, ...,    0.      ,
           0.      ,    0.      ]], dtype=float32)